Possible Datasets: (landsat loaded rn)

1. [Sparcs Dataset ~2GB](https://www.usgs.gov/core-science-systems/nli/landsat/spatial-procedures-automated-removal-cloud-and-shadow-sparcs)

2. [Landsat Validation Data ~100GB](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-8-cloud-cover-assessment-validation-data?qt-science_support_page_related_con=1#qt-science_support_page_related_con)

3. [Kaggle Dataset ~20GB](https://www.kaggle.com/sorour/95cloud-cloud-segmentation-on-satellite-images)

In [196]:
from bs4 import BeautifulSoup
from pathlib import Path
import requests

import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_datasets as tfds
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np

In [2]:
# landsat 8 data, should be 96 files of the following form:
# 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz'

url = 'https://landsat.usgs.gov/landsat-8-cloud-cover-assessment-validation-data'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
links = []
for link in soup.find_all('a'):
    link = link.get('href')
    criterion1 = 'https://landsat.usgs.gov/cloud-validation/cca_l8/'
    criterion2 = 'tar.gz'
    if link is not None and criterion1 in link and criterion2 in link:
      links.append(link)

print(len(links), links)

96 ['https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC80500092014231LGN00.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC80530022014156LGN00.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC81330312013202LGN00.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC81360302014162LGN00.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC81390292014135LGN00.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC81550082014263LGN00.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC81570452014213LGN00.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC81640502013179LGN01.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC81750432013144LGN00.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC81930452013126LGN01.tar.gz', 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC81990402014267LGN00.tar.gz', 'https://landsat.usgs.go

In [ ]:
# #### DOWNLOAD THE LANDDSAT 8 DATASET ####
def download(data_url):
  dl_manager = tfds.download.DownloadManager(download_dir='/tmp/junk/', extract_dir='/content/clouds')
  dataset_path = dl_manager.download_and_extract(data_url)
  return dataset_path

# creates a dataset consisting of image file paths
cloud_paths = download(links)

In [ ]:
# bands 2,3,4 are BGR (in that order)
# according to https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites?qt-news_science_products=0#qt-news_science_products

ds = []
for asset in cloud_paths:
  blue, green, red, mask = None, None, None, None
  paths = [str(path) for path in Path(asset).rglob('*_*')]
  for path in paths:
    if 'B2' in path: blue = path
    elif 'B3' in path: green = path
    elif 'B4' in path: red = path
    elif 'fixedmask.img' in path: 
      mask = path.replace('.img', '.TIF')
      # the usgs stored it in wack format, we have to call a command line
      # utility to convert from the mask file from .img to .TIF
      !gdal_translate -of GTiff {path + " " + mask}

  images = (red, green, blue, mask)
  # some times download gets corrupted, if so, throw that row away
  if None in images: pass
  else: ds.append(images)

# dataset where each row is a tuple of 
# paths for red, green, blue, and mask
# where each path points to a .TIF file
ds = tf.data.Dataset.from_tensor_slices(ds)
print(ds)

In [188]:
# convenience key-word args to parallel process
parallelize = dict(
  num_parallel_calls=tf.data.AUTOTUNE,
  deterministic=False)

# 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz'
# the mask values correspond to the following classes:
# 0	   Fill
# 64	   Cloud Shadow
# 128	   Clear
# 192	   Thin Cloud
# 255	   Cloud

#### READ IMAGE & MASK TO DATASET ####
@tf.function
def read_img_and_mask(files):
    # read img at specified path
    r, g, b, mask = files[0], files[1], files[2], files[3]
    r, g, b, mask = map(
        lambda x: tfio.experimental.image.decode_tiff(tf.io.read_file(x))[:, :, 0], 
        [r, g, b, mask])
    img = tf.experimental.numpy.dstack((r, g, b))

    thin_cloud = tf.where(mask == 192, True, False)
    cloud = tf.where(mask == 255, True, False)
    mask = tf.math.logical_or(thin_cloud, cloud)

    return img, mask

ds = ds.map(read_img_and_mask, **parallelize)
CARDINALITY = ds.cardinality()

In [189]:
# take n random crops of size h, w from an image and its mask 
@tf.function
def sample_crop(img, mask, h, w, n):
  # stack mask and image s.t. we crop both at the same time
  img_and_mask = tf.experimental.numpy.dstack((img, mask))
  crops = [tf.image.random_crop(img_and_mask, (h, w, 4)) for i in range(n)]
  # convert the n stacked, cropped images into a dataset
  crops = tf.stack(crops)
  crops = tf.data.Dataset.from_tensor_slices(crops)
  return crops

In [190]:
# randomly crop each img (and its mask) several times
# the result of calling this func on every row will return a new dataset
# that is n times the original cardinality
n, h, w, c = 20, 128, 128, 3
ds = ds.interleave(lambda img, mask: sample_crop(img, mask, h, w, n), **parallelize)

# tf doesn't know cardinality after interleave, so we help it out
CARDINALITY *= n
ds = ds.apply(tf.data.experimental.assert_cardinality(CARDINALITY))

In [191]:
# represent as tuple of img, mask rather than mask on image
# first three layers are the image, fourth layer is the mask
ds = ds.map(lambda x: (x[:, :, 0:3], x[:, :, 3]), **parallelize)

In [ ]:
# UNET ARCHITECTURE
# see original UNET paper here: https://arxiv.org/abs/1505.04597
# contact yash if you have questions about UNET
# TODO: maybe we could make this simpler while retaining most of the accuracy

# Build the model
inputs = layers.Input((h, w, c))
s = preprocessing.Rescaling(1.0 / 255)(inputs)

# Contraction path
c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(s)
c1 = layers.Dropout(0.1)(c1)
c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
p1 = layers.MaxPooling2D((2, 2))(c1)

c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
c2 = layers.Dropout(0.1)(c2)
c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
p2 = layers.MaxPooling2D((2, 2))(c2)
 
c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
c3 = layers.Dropout(0.2)(c3)
c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
p3 = layers.MaxPooling2D((2, 2))(c3)
 
c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
c4 = layers.Dropout(0.2)(c4)
c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c4)
p4 = layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p4)
c5 = layers.Dropout(0.3)(c5)
c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

#Expansive path 
u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = layers.concatenate([u6, c4])
c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
c6 = layers.Dropout(0.2)(c6)
c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)
 
u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = layers.concatenate([u7, c3])
c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
c7 = layers.Dropout(0.2)(c7)
c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)
 
u8 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = layers.concatenate([u8, c2])
c8 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u8)
c8 = layers.Dropout(0.1)(c8)
c8 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c8)
 
u9 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = layers.concatenate([u9, c1], axis=3)
c9 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u9)
c9 = layers.Dropout(0.1)(c9)
c9 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c9)
 
outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [198]:
# TODO: make a proper validation split

# 80-20 train-test split
BATCH_SIZE = 32
ds = ds.shuffle(buffer_size=CARDINALITY)
test_ds = ds.take(CARDINALITY // 5).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
train_ds = ds.skip(CARDINALITY // 5).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# once the val loss stops improving, stop and save 
# the best model we have created so far
callback = tf.keras.callbacks.EarlyStopping(
    restore_best_weights=True,
    patience=5
)

# TODO: change validation_data to valid_ds
history = model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=50,
  callbacks=[callback]
)

In [ ]:
model.evaluate(test_ds)